In [357]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import clear_output

In [346]:
import requests
import random
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]


In [347]:
import pickle #for saving output files, pickles
import time #for time.sleep function to delay calls
from tqdm import tqdm #for updating loop
#from os import listdir
#from os.path import isfile, join
import glob #pattern matching and expansion.

In [348]:
def proposal_process(proplist):
    '''takes a list with proposal info and processes some info into a df '''
    #we can use values in these keys 'as is'
    keeperkeys = ['id', 'proposalURL', 'fundURL', 'imageURL', 'retinaImageURL', 'thumbImageURL', 'fulfillmentTrailer', 'percentFunded', 'numDonors', 'costToComplete', 'studentLed', 'numStudents', 'professionalDevelopment', 'distanceLearningProject', 'totalPrice', 'freeShipping', 'teacherId', 'teacherName', 'teacherTypes', 'schoolTypes', 'schoolName', 'schoolUrl', 'city', 'zip', 'state', 'stateFullName', 'latitude', 'longitude', 'expirationDate', 'expirationTime', 'fundingStatus', 'fullyFundedDate', 'waitingForCheckPayment', 'modifiedDate','shortDescription']
    
    #these require more finagling
    moreprocess = ['matchingFund', 'gradeLevel', 'povertyType']
    
    combine=[]
    
    for prop in proplist:
        new_dict = {k: prop[k] for k in keeperkeys if k in prop}
        keepinfo = pd.DataFrame.from_dict(new_dict,orient='index').T
        
        if(not bool(prop['matchingFund'].get('matchingKey'))):
            keepinfo['matchingfund'] = 'no'
        else:
            keepinfo['matchingfund'] = 'no'
            
        keepinfo['grade_level'] = prop['gradeLevel']['name']
        keepinfo['poverty_level'] = prop['povertyType']['label']
        combine.append(keepinfo)
        
    summarydf = pd.concat(combine)
    return(summarydf)

In [382]:
tenner = list(range(0,50,10))

for p in range(len(tenner)):
    print(tenner.pop(random.randrange(len(tenner))))
    
print(tenner)

20
40
10
30
0
[]


In [383]:
# 1,674,610 completed projects 
prog_count = 0
iteration = 0
tenner = list(range(0,int(1674550/50),10))
#tenner = list(range(0,50,10))
historical = "https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&index="
out_file = "/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_"

In [371]:
  
while prog_count <= max(tenner):  #1674610
    
    #Pick a random user agent
    user_agent = random.choice(user_agent_list)
    #Set the headers 
    headers = {'User-Agent': user_agent}
         
    #randomly pops element from tenner out (for use), and removes it from tenner so it isn't resampled    
    random_element_from_tenner = tenner.pop(random.randrange(len(tenner)))     
        
    page_to_use = historical + str(random_element_from_tenner) #change index value every iteration, references tenner list

    #Make the request
    r = requests.get(page_to_use,headers=headers) #send request to specific html address
    
    try:
        data_dict = json.loads(r.text) #load the text as a dictionary
    except: #catch *all* exceptions
        tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
        continue #if we have an exception, *continue* will skip the rest of the iteration
    
    dicts_on_this_page = list(data_dict.values())
    proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
    proposal_list = dicts_on_this_page[proposal_ind]

    outframe = proposal_process(proposal_list)
    #print(outframe['teacherName'])
    pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
    pickle.dump(outframe, pickle_out)
    pickle_out.close()
    iteration += 1
    prog_count += 50
    print("Current progress:",np.round(prog_count/(max(tenner))),"%")
    time.sleep(random.randint(2,30))

https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&index=1010


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [390]:
user_agent = random.choice(user_agent_list)
#Set the headers 
headers = {'User-Agent': user_agent}

random_element_from_tenner = tenner.pop(random.randrange(len(tenner)))     

page_to_use = historical + str(random_element_from_tenner) #change index value every iteration, references tenner list

  

In [391]:
page_to_use

'https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&index=17570'

In [392]:
tenner


[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580, 1590, 1600, 1610, 1620, 1630, 1640, 1650, 1660, 1670, 1680, 1690, 1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800, 1810, 1820, 1830, 1840, 

In [373]:
iteration
page_to_use = historical + str(tenner[iteration]) #change index value every iteration, references tenner list
print(page_to_use)

https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&index=1010


In [376]:
r = requests.get(page_to_use) #send request to specific html address
r

<Response [403]>

In [374]:
data_dict = json.loads(r.text) #load the text as a dictionary
dicts_on_this_page = list(data_dict.values())
proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
proposal_list = dicts_on_this_page[proposal_ind]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [372]:
outframe = proposal_process(proposal_list)
#print(outframe['teacherName'])
pickle_out = open(out_file+("{0:03}".format(iteration))+'.pickle',"wb")
pickle.dump(outframe, pickle_out)
pickle_out.close()
iteration += 1
prog_count += 50
print("Current progress:",np.round(prog_count/(max(tenner))),"%")

101

In [301]:
#get all pickle files from folder into a list, 'picks'
picks = sorted(glob.glob("/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/*pickle")) 

In [302]:
#this will create a master list with the same # of elements as projects
basic_list = []
for pf in picks:
    print(pf)
    basic_list.append(pickle.load(open(pf,"rb")))
    #basic_list = basic_list+(pickle.load(open(picks[0],"rb")))

/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_000.pickle
/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_001.pickle
/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_002.pickle
/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_003.pickle


In [307]:
one=pickle.load(open('/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_000.pickle',"rb"))
two=pickle.load(open('/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_001.pickle',"rb"))
three=pickle.load(open('/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_002.pickle',"rb"))
four=pickle.load(open('/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_003.pickle',"rb"))

In [308]:
one

,id,proposalURL,fundURL,imageURL,retinaImageURL,thumbImageURL,fulfillmentTrailer,percentFunded,numDonors,costToComplete,studentLed,numStudents,professionalDevelopment,distanceLearningProject,totalPrice,freeShipping,teacherId,teacherName,teacherTypes,schoolTypes,schoolName,schoolUrl,city,zip,state,stateFullName,latitude,longitude,expirationDate,expirationTime,fundingStatus,fullyFundedDate,waitingForCheckPayment,modifiedDate,shortDescription,matchingfund,grade_level,poverty_level
0,4915422,https://www.donorschoose.org/project/math-bind...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,Help me give my students good binders to use i...,100,8,0.00,False,155,False,True,304.88,true,2370025,Mrs. Fitzhugh,"[{'id': 9, 'name': 'NEA member'}, {'id': 5, 'n...","[{'id': 12, 'name': 'Horace Mann (partner)'}]",Pierre S DuPont Middle School,https://www.donorschoose.org/school/pierres-du...,Wilmington,19802-2699,DE,Delaware,39.763720000000000,-75.533808000000000,2020-09-27,1601179200000,funded,"Thursday, May 28, 2020 3:39:10 PM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",Our school environment completely changed on M...,no,Grades 6-8,HIGH
0,4841284,https://www.donorschoose.org/project/rainbow-p...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,Help me give my students Physical Education eq...,100,1,0.00,False,26,False,False,248.76,false,107144,Mrs. Voegele,"[{'id': 9, 'name': 'NEA member'}, {'id': 5, 'n...","[{'id': 12, 'name': 'Horace Mann (partner)'}]",Jackson Avenue Elementary School,https://www.donorschoose.org/school/jackson-av...,Livermore,94551,CA,California,37.688380299999999,-121.791653699999998,2020-07-08,1594180800000,funded,"Tuesday, March 10, 2020 1:37:29 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",I teach kindergarten in a Title I Northern Cal...,no,Grades PreK-2,MODERATEHIGH
0,4918858,https://www.donorschoose.org/project/help-us-l...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u37...,https://www.donorschoose.org/teacher/photo/u37...,https://www.donorschoose.org/teacher/photo/u37...,Help me give my students the materials and sup...,100,5,0.00,False,20,False,True,599.45,false,3756874,Ms. H.,[],"[{'id': 12, 'name': 'Horace Mann (partner)'}]",Jefferson Elementary School,https://www.donorschoose.org/school/jefferson-...,Erie,16504-1597,PA,Pennsylvania,42.103943000000000,-80.062529000000000,2020-09-09,1599624000000,funded,"Thursday, May 21, 2020 7:27:03 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",My second grade students are adapting to learn...,no,Grades PreK-2,HIGHEST
0,4949656,https://www.donorschoose.org/project/building-...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u40...,https://www.donorschoose.org/teacher/photo/u40...,https://www.donorschoose.org/teacher/photo/u40...,Help me give my students access to resources f...,100,7,0.00,False,12,False,True,393.53,true,4006295,Mrs. Parnell,"[{'id': 5, 'name': 'SONIC Teacher'}]",[],Chester Jordan Elementary School,https://www.donorschoose.org/school/chester-jo...,El Paso,79938-5399,TX,Texas,31.801523000000000,-106.234489000000000,2020-09-28,1601265600000,funded,"Friday, May 29, 2020 12:42:02 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",I am a special education teacher that works wi...,no,Grades 3-5,MODERATEHIGH
0,4815720,https://www.donorschoose.org/project/diverse-b...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,Help me give my students picture books biograp...,100,2,0.00,False,850,False,False,

In [309]:
two

,id,proposalURL,fundURL,imageURL,retinaImageURL,thumbImageURL,fulfillmentTrailer,percentFunded,numDonors,costToComplete,studentLed,numStudents,professionalDevelopment,distanceLearningProject,totalPrice,freeShipping,teacherId,teacherName,teacherTypes,schoolTypes,schoolName,schoolUrl,city,zip,state,stateFullName,latitude,longitude,expirationDate,expirationTime,fundingStatus,fullyFundedDate,waitingForCheckPayment,modifiedDate,shortDescription,matchingfund,grade_level,poverty_level
0,4926874,https://www.donorschoose.org/project/virtual-l...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u57...,https://www.donorschoose.org/teacher/photo/u57...,https://www.donorschoose.org/teacher/photo/u57...,Help me give my students resources for distanc...,100,2,0.00,False,24,False,True,165.47,true,577821,Mrs. Fimbres,[],[],Peres K-8 School,https://www.donorschoose.org/school/peres-elem...,Richmond,94801-2654,CA,California,37.943016000000000,-122.364521000000000,2020-09-14,1600056000000,funded,"Tuesday, June 2, 2020 8:03:17 PM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",My students come from low-income areas and low...,no,Grades PreK-2,HIGHEST
0,4920443,https://www.donorschoose.org/project/oops-the-...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,Help me give my students math essential suppli...,100,4,0.00,False,33,False,True,324.48,true,2386144,Ms. MacDonald,"[{'id': 11, 'name': 'U.S. Cellular teacher'}, ...",[],Ben Painter Elementary School,https://www.donorschoose.org/school/ben-painte...,San Jose,95133-2099,CA,California,37.148140221834183,-119.644248783588410,2020-09-11,1599796800000,funded,"Friday, May 29, 2020 12:38:21 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT","My students are the most curious, intrigued, e...",no,Grades 3-5,HIGH
0,4741437,https://www.donorschoose.org/project/listen-lo...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u45...,https://www.donorschoose.org/teacher/photo/u45...,https://www.donorschoose.org/teacher/photo/u45...,Help me give my students an engaging listening...,100,10,0.00,False,21,False,False,447.88,true,4578418,Mrs. Bean,"[{'id': 9, 'name': 'NEA member'}]","[{'id': 12, 'name': 'Horace Mann (partner)'}]",Woodbridge Early Childhood Education Center,https://www.donorschoose.org/school/woodbridge...,Greenwood,19950,DE,Delaware,38.804687500000000,-75.587135314941400,2020-06-06,1591416000000,funded,"Tuesday, June 2, 2020 8:01:22 PM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",As a teacher in a low-income/high poverty scho...,no,Grades PreK-2,HIGH
0,4477141,https://www.donorschoose.org/project/help-make...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u47...,https://www.donorschoose.org/teacher/photo/u47...,https://www.donorschoose.org/teacher/photo/u47...,Help me give my students more computers in ord...,100,10,0.00,False,20,False,False,1378.64,true,4765366,Teacher Pinger,[],"[{'id': 12, 'name': 'Horace Mann (partner)'}, ...",Discovery School-Four Corners,https://www.donorschoose.org/school/four-corne...,Greenfield,01301-1416,MA,Massachusetts,42.592109000000000,-72.601783000000000,2020-01-13,1578891600000,funded,"Wednesday, January 8, 2020 2:28:26 PM EST",false,"Friday, March 13, 2020 12:00:00 AM EDT",Hungry to learn. Our 4th graders continually r...,no,Grades 3-5,MODERATELOW
0,4897198,https://www.donorschoose.org/project/the-show-...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u51...,https://www.donorschoose.org/teacher/photo/u51...,https://www.donorschoose.org/teacher/photo/u51...,Help me give my students tools that will allow...,100,6,0.00,False,200,False,True,549.18,true,51157,Mrs. C.,"[{'id': 9, 'name': 'NEA member'}]","[{'id': 12, 'name': 'Hor

In [303]:
bigframe = pd.concat(basic_list)

In [304]:
bigframe

,id,proposalURL,fundURL,imageURL,retinaImageURL,thumbImageURL,fulfillmentTrailer,percentFunded,numDonors,costToComplete,studentLed,numStudents,professionalDevelopment,distanceLearningProject,totalPrice,freeShipping,teacherId,teacherName,teacherTypes,schoolTypes,schoolName,schoolUrl,city,zip,state,stateFullName,latitude,longitude,expirationDate,expirationTime,fundingStatus,fullyFundedDate,waitingForCheckPayment,modifiedDate,shortDescription,matchingfund,grade_level,poverty_level
0,4915422,https://www.donorschoose.org/project/math-bind...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,Help me give my students good binders to use i...,100,8,0.00,False,155,False,True,304.88,true,2370025,Mrs. Fitzhugh,"[{'id': 9, 'name': 'NEA member'}, {'id': 5, 'n...","[{'id': 12, 'name': 'Horace Mann (partner)'}]",Pierre S DuPont Middle School,https://www.donorschoose.org/school/pierres-du...,Wilmington,19802-2699,DE,Delaware,39.763720000000000,-75.533808000000000,2020-09-27,1601179200000,funded,"Thursday, May 28, 2020 3:39:10 PM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",Our school environment completely changed on M...,no,Grades 6-8,HIGH
0,4841284,https://www.donorschoose.org/project/rainbow-p...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,Help me give my students Physical Education eq...,100,1,0.00,False,26,False,False,248.76,false,107144,Mrs. Voegele,"[{'id': 9, 'name': 'NEA member'}, {'id': 5, 'n...","[{'id': 12, 'name': 'Horace Mann (partner)'}]",Jackson Avenue Elementary School,https://www.donorschoose.org/school/jackson-av...,Livermore,94551,CA,California,37.688380299999999,-121.791653699999998,2020-07-08,1594180800000,funded,"Tuesday, March 10, 2020 1:37:29 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",I teach kindergarten in a Title I Northern Cal...,no,Grades PreK-2,MODERATEHIGH
0,4918858,https://www.donorschoose.org/project/help-us-l...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u37...,https://www.donorschoose.org/teacher/photo/u37...,https://www.donorschoose.org/teacher/photo/u37...,Help me give my students the materials and sup...,100,5,0.00,False,20,False,True,599.45,false,3756874,Ms. H.,[],"[{'id': 12, 'name': 'Horace Mann (partner)'}]",Jefferson Elementary School,https://www.donorschoose.org/school/jefferson-...,Erie,16504-1597,PA,Pennsylvania,42.103943000000000,-80.062529000000000,2020-09-09,1599624000000,funded,"Thursday, May 21, 2020 7:27:03 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",My second grade students are adapting to learn...,no,Grades PreK-2,HIGHEST
0,4949656,https://www.donorschoose.org/project/building-...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u40...,https://www.donorschoose.org/teacher/photo/u40...,https://www.donorschoose.org/teacher/photo/u40...,Help me give my students access to resources f...,100,7,0.00,False,12,False,True,393.53,true,4006295,Mrs. Parnell,"[{'id': 5, 'name': 'SONIC Teacher'}]",[],Chester Jordan Elementary School,https://www.donorschoose.org/school/chester-jo...,El Paso,79938-5399,TX,Texas,31.801523000000000,-106.234489000000000,2020-09-28,1601265600000,funded,"Friday, May 29, 2020 12:42:02 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",I am a special education teacher that works wi...,no,Grades 3-5,MODERATEHIGH
0,4815720,https://www.donorschoose.org/project/diverse-b...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,Help me give my students picture books biograp...,100,2,0.00,False,850,False,False,

In [299]:
bigframe.head()

,id,proposalURL,fundURL,imageURL,retinaImageURL,thumbImageURL,fulfillmentTrailer,percentFunded,numDonors,costToComplete,studentLed,numStudents,professionalDevelopment,distanceLearningProject,totalPrice,freeShipping,teacherId,teacherName,teacherTypes,schoolTypes,schoolName,schoolUrl,city,zip,state,stateFullName,latitude,longitude,expirationDate,expirationTime,fundingStatus,fullyFundedDate,waitingForCheckPayment,modifiedDate,shortDescription,matchingfund,grade_level,poverty_level
0,4915422,https://www.donorschoose.org/project/math-bind...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,https://www.donorschoose.org/teacher/photo/u23...,Help me give my students good binders to use i...,100,8,0.00,False,155,False,True,304.88,true,2370025,Mrs. Fitzhugh,"[{'id': 9, 'name': 'NEA member'}, {'id': 5, 'n...","[{'id': 12, 'name': 'Horace Mann (partner)'}]",Pierre S DuPont Middle School,https://www.donorschoose.org/school/pierres-du...,Wilmington,19802-2699,DE,Delaware,39.763720000000000,-75.533808000000000,2020-09-27,1601179200000,funded,"Thursday, May 28, 2020 3:39:10 PM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",Our school environment completely changed on M...,no,Grades 6-8,HIGH
0,4841284,https://www.donorschoose.org/project/rainbow-p...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,Help me give my students Physical Education eq...,100,1,0.00,False,26,False,False,248.76,false,107144,Mrs. Voegele,"[{'id': 9, 'name': 'NEA member'}, {'id': 5, 'n...","[{'id': 12, 'name': 'Horace Mann (partner)'}]",Jackson Avenue Elementary School,https://www.donorschoose.org/school/jackson-av...,Livermore,94551,CA,California,37.688380299999999,-121.791653699999998,2020-07-08,1594180800000,funded,"Tuesday, March 10, 2020 1:37:29 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",I teach kindergarten in a Title I Northern Cal...,no,Grades PreK-2,MODERATEHIGH
0,4918858,https://www.donorschoose.org/project/help-us-l...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u37...,https://www.donorschoose.org/teacher/photo/u37...,https://www.donorschoose.org/teacher/photo/u37...,Help me give my students the materials and sup...,100,5,0.00,False,20,False,True,599.45,false,3756874,Ms. H.,[],"[{'id': 12, 'name': 'Horace Mann (partner)'}]",Jefferson Elementary School,https://www.donorschoose.org/school/jefferson-...,Erie,16504-1597,PA,Pennsylvania,42.103943000000000,-80.062529000000000,2020-09-09,1599624000000,funded,"Thursday, May 21, 2020 7:27:03 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",My second grade students are adapting to learn...,no,Grades PreK-2,HIGHEST
0,4949656,https://www.donorschoose.org/project/building-...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u40...,https://www.donorschoose.org/teacher/photo/u40...,https://www.donorschoose.org/teacher/photo/u40...,Help me give my students access to resources f...,100,7,0.00,False,12,False,True,393.53,true,4006295,Mrs. Parnell,"[{'id': 5, 'name': 'SONIC Teacher'}]",[],Chester Jordan Elementary School,https://www.donorschoose.org/school/chester-jo...,El Paso,79938-5399,TX,Texas,31.801523000000000,-106.234489000000000,2020-09-28,1601265600000,funded,"Friday, May 29, 2020 12:42:02 AM EDT",false,"Wednesday, June 3, 2020 12:00:00 AM EDT",I am a special education teacher that works wi...,no,Grades 3-5,MODERATEHIGH
0,4815720,https://www.donorschoose.org/project/diverse-b...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,https://www.donorschoose.org/teacher/photo/u21...,Help me give my students picture books biograp...,100,2,0.00,False,850,False,False,

In [300]:
bigframe['proposalURL'].value_counts()

https://www.donorschoose.org/project/non-fiction-for-curious-minds/4225892/               4
https://www.donorschoose.org/project/soft-starts-to-increase-focus/4630880/               4
https://www.donorschoose.org/project/new-fossil-plate-kits-for-students-to-ex/4957790/    4
https://www.donorschoose.org/project/putting-it-all-together/4947967/                     4
https://www.donorschoose.org/project/sports-for-physical-education/4873326/               4
https://www.donorschoose.org/project/curious-minds-love-stem-science-the-hum/4890556/     4
https://www.donorschoose.org/project/feeling-all-the-feelings-make-social-e/4510532/      4
https://www.donorschoose.org/project/tools-to-help-expand-creativity/4270556/             4
https://www.donorschoose.org/project/magic-of-music/4661625/                              4
https://www.donorschoose.org/project/super-sensory-walk/4843498/                          4
https://www.donorschoose.org/project/new-athletic-uniforms-for-middle-school/495

In [186]:
checkdf = pd.DataFrame(project_IDs,columns =['proj_id'])
checkdf
pd.set_option("max_rows", None) #undo by resetting --- pd.reset_option(“max_rows”)
checkdf

,proj_id
0,4770461
1,4822073
2,4912619
3,4873150
4,4904035
5,4908254
6,4890517
7,4955536
8,4937550
9,4915416


In [182]:
checkdf['proj_id'].value_counts()

4885359    4
4918039    4
4879853    4
4764841    4
4956913    4
4475625    4
4947668    4
4908254    4
4945703    4
4770584    4
4732489    4
4904035    4
4902361    4
4953575    4
4944722    4
4949899    4
4948100    4
4943826    4
4796421    4
4320160    4
4955536    4
4814839    4
4939356    4
4621985    4
4904970    4
4947428    4
4931271    4
4654114    4
4915416    4
4947119    4
4912619    4
4806437    4
4167102    4
4935614    4
4770461    4
4879494    4
4880636    4
4161759    4
4535151    4
4525325    4
4873150    4
4759273    4
4408750    4
4937550    4
4822073    4
4944540    4
4664368    4
4811184    4
4890517    4
4805579    4
Name: proj_id, dtype: int64

In [166]:
basic_list[0]

{'id': '4770461', 'proposalURL': 'https://www.donorschoose.org/project/inspiring-students/4770461/', 'fundURL': 'https://secure.donorschoose.org/donors/givingCart.html?proposalid=4770461&donationAmount=45', 'imageURL': 'https://www.donorschoose.org/teacher/photo/u3690019?size=sm&t=1532720007365', 'retinaImageURL': 'https://www.donorschoose.org/teacher/photo/u3690019?size=retina&t=1532720007365', 'thumbImageURL': 'https://www.donorschoose.org/teacher/photo/u3690019?size=thmb&t=1532720007365', 'title': 'Inspiring Students', 'shortDescription': 'Teaching first grade brings joy to me each day. My students come from a variety of backgrounds in West Philadelphia. These backgrounds melt together when we are in class because we are able to...', 'fulfillmentTrailer': 'Help me give my students Reading Rods, ink cartridges, hand sanitizer, and more.', 'snippets': [], 'percentFunded': '100', 'numDonors': '3', 'costToComplete': '0.00', 'studentLed': False, 'numStudents': '30', 'professionalDevelopm

In [147]:
bewild = pd.DataFrame.from_dict(lookat)

ValueError: arrays must all be same length

In [45]:
import json
from bs4 import BeautifulSoup

In [46]:
import scrapy
#modified scrapy settings here:
#/home/russell/anaconda3/envs/insight/lib/python3.8/site-packages/scrapy/settings
#to include the user agents described here: https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/

In [47]:
df = pd.read_csv("/home/russell/Downloads/opendata_projects.csv", thousands = ',')
pd.set_option('display.max_columns', None)
#pd.set_option("max_rows", None) #undo by resetting --- pd.reset_option(“max_rows”)bb

In [101]:
current = "https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=100&index=0"
historical = "https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=40&historical=true&index=0"


In [102]:
r = requests.get(historical)
data_dict = json.loads(r.text)

In [72]:
print(data_dict)

{'searchTerms': 'Most urgent', 'searchURL': 'https://www.donorschoose.org/donors/search.html?historical=true', 'searchVOHashCode': 1547686084, 'totalProposals': '1674618', 'index': '0', 'max': '50', 'breadcrumb': [['max', '100', ''], ['historical', 'true', 'Completed']], 'proposals': [{'id': '4944919', 'proposalURL': 'https://www.donorschoose.org/project/help-care-for-me-at-home-please/4944919/', 'fundURL': 'https://secure.donorschoose.org/donors/givingCart.html?proposalid=4944919&donationAmount=45', 'imageURL': 'https://www.donorschoose.org/teacher/photo/u1616220?size=sm&t=1394165327649', 'retinaImageURL': 'https://www.donorschoose.org/teacher/photo/u1616220?size=retina&t=1394165327649', 'thumbImageURL': 'https://www.donorschoose.org/teacher/photo/u1616220?size=thmb&t=1394165327649', 'title': 'Help Care For Me At Home Please', 'shortDescription': 'My students are the best inner-city kids. Throughout this whole pandemic, I have slowly watched them lose their smiles and become depressed

In [98]:
for key in data_dict.items():
    print (key)

('searchTerms', 'Most urgent')
('searchURL', 'https://www.donorschoose.org/donors/search.html?historical=true')
('searchVOHashCode', 1547686084)
('totalProposals', '1674619')
('index', '0')
('max', '50')
('breadcrumb', [['max', '100', ''], ['historical', 'true', 'Completed']])
('proposals', [{'id': '4890670', 'proposalURL': 'https://www.donorschoose.org/project/writing-in-colors-from-the-distance/4890670/', 'fundURL': 'https://secure.donorschoose.org/donors/givingCart.html?proposalid=4890670&donationAmount=45', 'imageURL': 'https://www.donorschoose.org/teacher/photo/u326209?size=sm&t=1533943300307', 'retinaImageURL': 'https://www.donorschoose.org/teacher/photo/u326209?size=retina&t=1533943300307', 'thumbImageURL': 'https://www.donorschoose.org/teacher/photo/u326209?size=thmb&t=1533943300307', 'title': 'Writing In Colors From The Distance', 'shortDescription': 'My students have been out of school since mid March. We left on spring break and never returned. We did not know we were not re

In [103]:
dicts_on_this_page = list(data_dict.values())

proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #

proposal_list = dicts_on_this_page[proposal_ind]
#dicts_on_this_page[proposal_ind]
#proposal_ind = which(data.keys()=='proposals')
#print(proposal_ind)
#print(proposals_on_this_page[7])

In [116]:
proposal_list[0]

{'id': '4928948', 'proposalURL': 'https://www.donorschoose.org/project/distance-learning-reading-and-math-educ/4928948/', 'fundURL': 'https://secure.donorschoose.org/donors/givingCart.html?proposalid=4928948&donationAmount=45', 'imageURL': 'https://www.donorschoose.org/teacher/photo/u1016971?size=sm&t=1325728301000', 'retinaImageURL': 'https://www.donorschoose.org/teacher/photo/u1016971?size=retina&t=1325728301000', 'thumbImageURL': 'https://www.donorschoose.org/teacher/photo/u1016971?size=thmb&t=1325728301000', 'title': 'Distance Learning Reading and Math- Education Is Essential!', 'shortDescription': 'My students are the most creative, active, and hard working learners. Many of them come from low-income homes and lack the resources they need to be successful. They have family members working on...', 'fulfillmentTrailer': 'Help me give my students math manipulatives and materials, books, and writing workbooks to help them at home.', 'snippets': [], 'percentFunded': '100', 'numDonors':

In [112]:
first_proposal = proposal_list[0] #returns dictionary of first proposal items

In [113]:
type(first_proposal)

<class 'dict'>

In [95]:
for key in first_proposal.items():
    print (key)

('id', '4944919')
('proposalURL', 'https://www.donorschoose.org/project/help-care-for-me-at-home-please/4944919/')
('fundURL', 'https://secure.donorschoose.org/donors/givingCart.html?proposalid=4944919&donationAmount=45')
('imageURL', 'https://www.donorschoose.org/teacher/photo/u1616220?size=sm&t=1394165327649')
('retinaImageURL', 'https://www.donorschoose.org/teacher/photo/u1616220?size=retina&t=1394165327649')
('thumbImageURL', 'https://www.donorschoose.org/teacher/photo/u1616220?size=thmb&t=1394165327649')
('title', 'Help Care For Me At Home Please')
('shortDescription', 'My students are the best inner-city kids. Throughout this whole pandemic, I have slowly watched them lose their smiles and become depressed. They miss school, and they miss their friends. I want...')
('fulfillmentTrailer', 'Help me give my students hygiene items and healthier snacks for their homes.')
('snippets', [])
('percentFunded', '100')
('numDonors', '2')
('costToComplete', '0.00')
('studentLed', False)
('num

In [44]:
print(soup)

{
  "searchTerms": "Most urgent",
  "searchURL": "https://www.donorschoose.org/donors/search.html?historical=true",
  "searchVOHashCode": 1547686084,
  "totalProposals": "1674603",
  "index": "0",
  "max": "50",
  "breadcrumb": [
    [
      "max",
      "100",
      ""
    ],
    [
      "historical",
      "true",
      "Completed"
    ]
  ],
  "proposals": [
    {
      "id": "4855975",
      "proposalURL": "https://www.donorschoose.org/project/books-to-keep-students-reading-during-co/4855975/",
      "fundURL": "https://secure.donorschoose.org/donors/givingCart.html?proposalid=4855975&amp;donationAmount=45",
      "imageURL": "https://www.donorschoose.org/teacher/photo/u11668?size=sm&amp;t=1545428762141",
      "retinaImageURL": "https://www.donorschoose.org/teacher/photo/u11668?size=retina&amp;t=1545428762141",
      "thumbImageURL": "https://www.donorschoose.org/teacher/photo/u11668?size=thmb&amp;t=1545428762141",
      "title": "Books to Keep Students Reading During Covid-19 Cri

# _DRAW OUT SCREENS and FUNCTIONALITY GOALS FOR END OF WEEK_



## MONDAY
    Well scoped, clearly-defined problem + some data
## 	TUESDAY
    SQL[organized data] 
## 	WED 
    analytics/working algo/some results ----> (python linked to sql data)
## 	THUR – SQL mapped to PYTHON connected to FLASK (or something)


# GOOD Qs to ASK SELF AND OTHERS
#### 	-What’s actionable about your product?
#### 	-Did you try other models?
#### 	-Is this better than random?
#### 	-Is this better than the simplest model?
#### 	-Why did you choose these inputs?
#### 	-How did you validate this?
#### 	-What are your metrics for success?
#### 	-What are the assumptions of your model?
#### 	-How would you improve this project with more time?

In [3]:
df.shape

(1203287, 44)

In [4]:
df.head()

,_projectid,_teacher_acctid,_schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,"""teacher_ny_teaching_fellow""",primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,vendor_shipping_charges,sales_tax,payment_processing_charges,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,total_donations,num_donors,eligible_double_your_impact_match,eligible_almost_home_match,funding_status,date_posted,date_completed,date_thank_you_packet_mailed,date_expiration
0,7342bd01a2a7725ce033a179d22e382d,5c43ef5eac0f5857c266baa1ccfa3d3f,9e72d6f2f1e9367b578b6479aa5852b7,3.600097e+11,40.688454,-73.910432,New York City,NY,11207.0,urban,New York City Dept Of Ed,Kings (Brooklyn),f,t,f,f,f,f,Mr.,f,f,Other,Applied Learning,NaN,NaN,Supplies,highest poverty,Grades 6-8,NaN,NaN,NaN,NaN,229.0,279.27,0.0,251.9,1,f,f,completed,2002-09-13 00:00:00,2002-09-23 00:00:00,2003-01-27 00:00:00,2003-12-31 00:00:00
1,ed87d61cef7fda668ae70be7e0c6cebf,1f4493b3d3fe4a611f3f4d21a249376a,1ae4695be589a36816188e2b301a0395,3.600077e+11,40.765517,-73.960090,New York City,NY,10065.0,NaN,New York City Dept Of Ed,New York (Manhattan),f,f,f,f,f,f,Ms.,f,f,Literacy,Literacy & Language,NaN,NaN,Books,highest poverty,Grades 9-12,NaN,NaN,NaN,NaN,125.0,152.44,0.0,137.0,1,f,f,completed,2002-09-13 00:00:00,2002-09-23 00:00:00,2003-01-03 00:00:00,2003-12-31 00:00:00
2,b56b502d25666e29550d107bf7e17910,57426949b47700ccf62098e1e9b0220c,4a06a328dd87bd29892d73310052f45f,3.600077e+11,40.770233,-73.950760,New York City,NY,10075.0,NaN,New York City Dept Of Ed,New York (Manhattan),f,t,f,f,f,f,Ms.,f,f,Early Development,Applied Learning,NaN,NaN,Supplies,moderate poverty,Grades 3-5,NaN,NaN,NaN,NaN,125.0,152.44,0.0,125.0,1,f,f,completed,2002-09-16 00:00:00,2002-09-19 00:00:00,2002-12-19 00:00:00,2003-12-31 00:00:00
3,016f03312995d5c89d6b348be4682166,9c0aa56b63b743454d6da9effcf122fc,bb0af5dac1b54693ba86ef63eacd6594,3.600076e+11,40.727826,-73.978721,New York City,NY,10009.0,urban,New York City Dept Of Ed,New York (Manhattan),f,f,f,f,f,f,Ms.,f,f,History & Geography,History & Civics,NaN,NaN,Supplies,high poverty,Grades PreK-2,NaN,NaN,NaN,NaN,187.0,228.05,0.0,205.0,1,f,f,completed,2002-09-17 00:00:00,2002-09-17 00:00:00,2002-12-02 00:00:00,2003-12-31 00:00:00
4,cf6275558534ca1b276b0d8d5130dd9a,1d4d8a42730dbb66af1ebb6ab37456b7,768dab263f87881fe7c68ffb3965df7c,3.600083e+11,40.841216,-73.938605,New York City,NY,10032.0,urban,New York City Dept Of Ed,New York (Manhattan),f,f,f,f,f,f,Ms.,t,f,Other,Applied Learning,NaN,NaN,Supplies,highest poverty,Grades PreK-2,NaN,NaN,NaN,NaN,240.0,292.68,0.0,264.0,1,f,f,completed,2002-09-17 00:00:00,2002-09-23 00:00:00,2003-02-26 00:00:00,2003-12-31 00:00:00


In [5]:
df.columns

Index(['_projectid', '_teacher_acctid', '_schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_zip', 'school_metro', 'school_district', 'school_county',
       'school_charter', 'school_magnet', 'school_year_round', 'school_nlns',
       'school_kipp', 'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', ' "teacher_ny_teaching_fellow"',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'vendor_shipping_charges', 'sales_tax',
       'payment_processing_charges', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'total_donations', 'num_donors', 'eligible_double_your_impact_match',
       'eligible_almost_home_match', 'funding_status', 'date_posted',
       'date_complet

In [18]:
df['primary_focus_area'].value_counts()

Literacy & Language    508084
Math & Science         326323
Music & The Arts       105410
Applied Learning        80312
Special Needs           78884
Health & Sports         53519
History & Civics        50709
Name: primary_focus_area, dtype: int64

In [16]:
expired = df[(df['funding_status']=='expired')]

In [17]:
df['_teacher_acctid'].value_counts()

fa2f220b537e8653fb48878ebb38044d    370
a006826c170f91f85ff80dc5a132fade    361
02bccf5c109ace4f3dcbce819a46daa1    349
9d7051e2611cebdb758f1c7bd09360ac    316
9f49ba20aa1c28eb95dbad8b8edd2b69    316
                                   ... 
899ce20468db07d649a58e38a25ed55d      1
266bd98ea7e71bf1fbad5a0dd8ec69f0      1
afc712b464db278714755cee218caad2      1
f10ae341471a4cb05db7d90c7fb82301      1
43569fe13dc7c6c827213a5c2d3d6223      1
Name: _teacher_acctid, Length: 423664, dtype: int64